# Setup

```
docker compose -f devops/docker/docker-compose.yaml up -d qdrant
```

# Imports

In [11]:
import packages

from context.utils import typer as t

from toolkit.utils import utils
from toolkit.utils.utils import rp_print

from context.external import clients
import context.instances as inst
import context.consts as const
import context.settings.main as settings_main

from toolkit.llm.langchain.core import integration, utils as utils_lc
from toolkit.llm.langchain.data.persistence import retrievers
from toolkit.llm.langchain.data.indexing import (
  documents, document_loaders, text_splitters,
)
from toolkit.llm.langchain.execution import (
  runnables, graphs, tools as tools_lc, agents
)
from toolkit.llm.langchain.models import (
	prompts as prompts_lc, llms, messages as msgs_lc
)

In [2]:
# vector_store = inst.vector_store_in_memory
vector_stores = inst.vector_stores_qdrant

COLLS = settings_main.VEC_STR_COLLS

llm = inst.llm_main
embedding = inst.embedding_main

# Dev

## <span style="color: #f59e0b;">Load</span>
<span style="color: red;">Only run once</span>

In [ ]:
# document_sample = documents.Document(
#     page_content="content",
#     metadata={"key": "value"},
# )

# docs = [document_sample]

In [3]:
loader = document_loaders.UnstructuredPDFLoader(
	file_path=f"{packages.APP_PATH}/data/org/manual_toyota_corolla_cross_2023.pdf",
)
docs = loader.load()

In [4]:
# https://python.langchain.com/docs/how_to/semantic-chunker/
text_splitter = text_splitters.SemanticChunker(
  embeddings=embedding
)
splits = text_splitter.split_documents(docs)

In [ ]:
# len_splits = len(splits)

# key = "something"
# value = "something"

# for i, doc in enumerate(splits):
# 	doc.metadata[key] = value

In [5]:
uuids = [str(documents.uuid4()) for _ in range(len(splits))]
document_ids = vector_stores[COLLS.CAR].add_documents(documents=splits, ids=uuids)

In [ ]:
# rp_print(docs)
# print(docs[0].page_content)
# len(docs)

# len(splits)

## Query

In [12]:
questions = [
	"How to Use a Seat Belt",
]

question = questions[0]

retriever = vector_stores[COLLS.CAR].as_retriever(
	search_type=retrievers.ModeRetriever.MMR,
	search_kwargs={
		"k": 3,
	}
)

# retrieved_docs = vector_stores[COLLS.CAR].similarity_search_with_score(
# 	query=question, k=5,
# )
retrieved_docs = retriever.invoke(question)

In [ ]:
rp_print(retrieved_docs)

# Ref

- Qdrant
  - https://qdrant.tech/documentation/guides/installation/
  - https://python.langchain.com/docs/integrations/vectorstores/qdrant/ ⌛
  - https://qdrant.tech/blog/qdrant-1.10.x/